In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('social_network_anonymized.db')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()

print (table_name)

# Read the entire database into a pandas DataFrame
media = pd.read_sql_query(f"SELECT * FROM Media", conn)
profile_activity = pd.read_sql_query(f"SELECT * FROM ProfileActivity", conn)
activity_media = pd.read_sql_query(f"SELECT * FROM ActivityMedia", conn)
profile_connection = pd.read_sql_query(f"SELECT * FROM ProfileConnection", conn)
profiles = pd.read_sql_query(f"SELECT * FROM Profiles", conn)
activity = pd.read_sql_query(f"SELECT * FROM Activity", conn)

# Close the connection
conn.close()

[('Media',), ('ProfileActivity',), ('ActivityMedia',), ('ProfileConnection',), ('Profiles',), ('Activity',)]


In [ ]:
def pair_profile_activities(profile_activity_df, profile_df):
    sources = profile_activity_df[profile_activity_df.relationship_type == "source"]
    targets = profile_activity_df[profile_activity_df.relationship_type == "target"]

    sources = pd.merge(
        sources,
        profile_df,
        left_on="profile_id",
        right_on="id"
    )

    targets = pd.merge(
        targets,
        profile_df,
        left_on="profile_id",
        right_on="id"
    )

    merged = pd.merge(
        sources,
        targets,
        on="activity_id",
        suffixes=("_source", "_target")
    )

    return merged

def merge_activities(df, activity_df, left_on):
    return pd.merge(
        df,
        activity_df,
        left_on=left_on,
        right_on="id"
    )

In [ ]:
profile_pairs = pair_profile_activities(profile_activity, profiles)
profile_pairs_activity = merge_activities(profile_pairs, activity, left_on="activity_id")

In [ ]:
profile_pairs_activity.columns

Index(['id_x_source', 'profile_id_source', 'activity_id',
       'relationship_type_source', 'id_y_source', 'name_source',
       'profile_type_source', 'platform_source', 'platform_id_source',
       'profile_url_source', 'region_source', 'id_x_target',
       'profile_id_target', 'relationship_type_target', 'id_y_target',
       'name_target', 'profile_type_target', 'platform_target',
       'platform_id_target', 'profile_url_target', 'region_target', 'id',
       'type', 'timestamp', 'content', 'description', 'platform'],
      dtype='object')